#### Importing Libs

In [1]:
from core.PipelineProject import PrepareDataAndTrainingModels
from core.SQLeto import SQLeto
from decouple import config
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

#### Load enviroment vars

In [2]:
RAW_DATA_PATH = config('RAW_DATA_PATH')
DATABASE_POSTGRES = config('DATABASE_POSTGRES')
USER_POSTGRES = config('USER_POSTGRES')
PWD_POSTGRES = config('PWD_POSTGRES')

#### Load data from POTSGRES

In [3]:
# Create object to access database
sqleto = SQLeto(
    database=DATABASE_POSTGRES, 
    user=USER_POSTGRES,
    password=PWD_POSTGRES)
    
# Getting data in a dataframe
data = sqleto.execute_DQL(
    '''
        SELECT * FROM creditcard
    '''
)

#### Load models

In [4]:
# Models to test
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

#### Load scaler

In [5]:
# Scaler
from sklearn.preprocessing import RobustScaler

#### Load balancer

In [6]:
# Balancer
from imblearn.over_sampling import SMOTE

#### Use Pipeline to avaliate Models

In [7]:
prep = PrepareDataAndTrainingModels(
    dataframe=data,
    target="Class",
    scaler=RobustScaler(),
    models=[
        DecisionTreeClassifier(),
        MLPClassifier(),
        LogisticRegression(),
        RandomForestClassifier(),
        GradientBoostingClassifier(),
        XGBClassifier()

    ],
    balancer = SMOTE(),
    score_metric = 'recall_macro'
)
prep.avaliate_models_cv()

Its Binary Classification!


##################################################


Using tecnique: SMOTE()
Model training: DecisionTreeClassifier()


